In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import defaultdict
from copy import deepcopy
import time
import random
from scipy import stats
from scipy.stats import mode
import seaborn as sns
import warnings
from collections import defaultdict
warnings.filterwarnings("ignore")
from scipy.stats import chisquare

In [2]:
!conda list -e >env.txt

## Extract potentiator hits for all antibiotics


In [3]:
# Merge sequential and simultaneous hits into a single table

all_hits_sequential = pd.read_csv("output/04A.pair_annotation/results_allpairs_annotation_withcategories.csv", index_col=0)
all_simultaneous = pd.read_csv("output/04B.pair_annotation/results_allpairs_annotation_withcategories.csv", index_col=0)

sequential_merged = all_hits_sequential.merge(all_simultaneous[["position_i", "position_j", "pval_beta_i_on_j", 'pval_beta_i_on_j_BH_sig']], 
                                                on=["position_i", "position_j"], how="left", suffixes=["_sequential", "_simultaneous"])

simultaneous_merged = all_simultaneous.merge(all_hits_sequential[["position_i", "position_j", "pval_beta_i_on_j", 'pval_beta_i_on_j_BH_sig']], 
                                                on=["position_i", "position_j"], how="left", suffixes=[ "_simultaneous", "_sequential"])

all_hits = pd.concat([simultaneous_merged, sequential_merged])
all_hits = all_hits.drop_duplicates(subset=["position_i", "position_j"]).reset_index(drop=True)



In [4]:
known_antibiotic_first = all_hits.dropna(subset=["known_drug_i"]).query("pval_beta_i_on_j_BH_sig_sequential or pval_beta_i_on_j_BH_sig_simultaneous")

known_antibiotic_first['num_dependent'] = known_antibiotic_first.n_muts_i_on_branch_j + known_antibiotic_first.n_muts_same_branch
know_ab_first = known_antibiotic_first.loc[known_antibiotic_first.possible_drug_i.isnull(), :]
know_ab_first.sort_values('num_dependent', ascending=False).head(40)

,Unnamed: 0,position_i,gene_name_i,position_j,gene_name_j,n_mutations_i,n_mutations_j,n_branches_with_i,n_branches_with_j,n_muts_i_on_branch_j,...,in_antigen_protein_j,in_known_ABR_position_j,known_drug_j,in_possible_ABR_gene_j,possible_drug_j,annotation,pval_beta_i_on_j_sequential,pval_beta_i_on_j_BH_sig_sequential,median_distance,num_dependent
527021,36273,1673425,None,1096633,None,786,5,3817,60683,768,...,False,False,NaN,False,NaN,known_AB_second,7.839476e-28,True,590.477720,768
528636,63830,1673425,None,2338994,Rv2082,786,19,3817,32513,491,...,False,False,NaN,False,NaN,known_AB_second,1.226391e-19,True,619.327023,491
527474,44432,1673425,None,1340208,PPE18,786,30,3817,21626,376,...,True,False,NaN,False,NaN,known_AB_second,3.146643e-22,True,559.401902,377
527709,48467,1673425,None,340132,PPE3,786,150,3817,21827,318,...,True,False,NaN,False,NaN,known_AB_second,1.889911e-05,True,568.675319,321
46692,5321,1673425,None,3883626,Rv3466,786,469,3817,19374,307,...,False,False,NaN,False,NaN,known_AB_second,2.470575e-08,True,NaN,316
587432,577848,1673425,None,2338990,Rv2082,786,18,3817,16336,282,...,False,False,NaN,False,NaN,known_AB_second,2.197887e-13,True,769.046309,282
55276,57354,1673425,None,3446699,Rv3081,786,83,3817,16441,207,...,False,False,NaN,False,NaN,known_AB_second,4.914138e-01,False,NaN,214
72583,120996,1673425,None,1722228,pks5,786,13,3817,16173,209,...,False,False,NaN,False,NaN,known_AB_second,2.981139e-01,False,NaN,212
48537,15793,1673425,None,3730411,PPE54,786,369,3817,15646,194,...,True,False,NaN,False,NaN,known_AB_second,4.264086e-01,False,NaN,211
589346,609374,1673425,None,1533583,PPE19,786,116,3817,10743,181,...,True,False,NaN,False,NaN,known_AB_second,1.493815e-07,True,603.102116,186


In [5]:
def _process_known_hits(input_list):
    """
    splits the possible associated drugs (string separated by ,) into a list
    """
    output_list = []
    for item in input_list:
        #print(item)
        if type(item) != str:
            output_list.append(np.nan)
        else:
            splitted = item.split(",")
            exclusive= list(set(splitted))
            output_list.append(exclusive)
    return output_list

## Note! This is code recycling, actually the table contains hits where the known antibiotic resistance mutation was second
known_antibiotic_first = all_hits.dropna(subset=["known_drug_i"]).query("pval_beta_i_on_j_BH_sig_sequential or pval_beta_i_on_j_BH_sig_simultaneous")

known_antibiotic_first["known_drug_i"] = _process_known_hits(known_antibiotic_first.known_drug_i)
known_antibiotic_first["possible_drug_i"] = _process_known_hits(known_antibiotic_first.possible_drug_i)

known_antibiotic_first["known_drug_j"] = _process_known_hits(known_antibiotic_first.known_drug_j)
known_antibiotic_first["possible_drug_j"] = _process_known_hits(known_antibiotic_first.possible_drug_j)

print(len(known_antibiotic_first))

5246


In [6]:
between_antibiotic_indices = []
antibiotic_indices = []

known_antibiotic_first["within_same_drug"] = False

for idx, row in known_antibiotic_first.iterrows():
    
    set_of_known_first = set(row.known_drug_i)
    set_of_known_second = set()
    known_drug_second = False
    
    if type(row.known_drug_j) is list:
        set_of_known_second= set(row.known_drug_j)
        known_drug_second = True
        
    elif type(row.possible_drug_j) is list:
        set_of_known_second= set(row.possible_drug_j)
        known_drug_second = True
        
    if not known_drug_second:
        antibiotic_indices.append(idx)
    
    elif len(set_of_known_first.intersection(set_of_known_second)) > 0:
        antibiotic_indices.append(idx)
        known_antibiotic_first.loc[idx, "within_same_drug"] = True
        
    else:
        between_antibiotic_indices.append(idx)
        
between_antibiotic_hits = known_antibiotic_first.loc[between_antibiotic_indices]
antibiotic_hits = known_antibiotic_first.loc[antibiotic_indices]
    

## Compute per-drug output tables

In [7]:
## These will contain all hits except the between-antibiotic hits

## First get the list of all names of drugs in the dataset
drug_list = []
for idx, row in antibiotic_hits.iterrows():
    drugs1 = row.possible_drug_i
    drugs2 = row.possible_drug_j
    
    if type(drugs1) != list:
        drugs1= []
    if type(drugs2) != list:
        drugs2=[]
    
    for drug in drugs1 + drugs2:
        drug_list.append(drug)

drug_list = list(set(drug_list))

## Then create an empty df for each
df_dict = {drug:pd.DataFrame() for drug in drug_list}

## now concatenate each row to the appropriate drug dataframe
for idx, row in antibiotic_hits.iterrows():
    drugs1 = row.possible_drug_i
    drugs2 = row.possible_drug_j
    
    if type(drugs1) != list:
        drugs1= []
    if type(drugs2) != list:
        drugs2=[]
    
    for drug in drugs1:
        df_dict[drug] = df_dict[drug].append(row)
        
    for drug in drugs2:
        df_dict[drug] = df_dict[drug].append(row)
        
for drug, df in df_dict.items():
    print(drug, len(df))
    df.sort_values("n_muts_i_on_branch_j", ascending=False).to_csv(f"output/05.antibiotic/{drug}_potentiator_antibiotic_hits.csv")
    df.query("pval_beta_i_on_j_BH_sig_sequential != 0").query("pval_beta_i_on_j_BH_sig_sequential").sort_values(
        "n_muts_i_on_branch_j", ascending=False
    ).to_csv(f"output/05.antibiotic/{drug}_potentiator_antibiotic_hits_sequential.csv")

ETH 209
EMB 348
LZD 221
LEV 358
AMI 424
KAN 417
RIF 581
INH 153
MXF 358
CAP 246
STM 497
PZA 1230
